## Homework #8
### Portfolio and Risk Management (Fall 2024)
### Brian Wickman

### 1. Conceptual Issues for LTCM

1. Describe LTCM's investment strategy wrt to securities traded, trading frequency, skewness (many small wins or a few big hits?), and forecasting (what is behind their selection of trades?).

LTCM primarily engaged in convergence and relative value trades with government bonds, low default risk corporate bonds and swaps, foreign currencies, and long-expiry options. The firm traded alot, with 100 different strategies and 7600 open positions during 1997, with medium-term investment horizon; convergence trades required time to reach equilibrium. LTCM sought to exploit small inefficiences through the use of leverage and cost-effective financing, typically resulting in many small wins, but also hit a few big wins. Trading opportunities were identified through security mispricings, statistical modelling, and forecasting market participant's liquidity needs.

2. What are LTCM's biggest advantages over its competitors?

LTCM was the valuation of novel financial instruments, the use of complex self-financing trades, and access to great financing options including a $700 million unsecured credit line and low haircuts as dealers competed for LTCM's business.

3. Briefly discuss LTCM's four types of fund risk (collateral haircuts, repo maturity, equity redemption, and loan access).

During periods of market stress, dealers increased collateral haircuts, which required LTCM to post more assets in a bad state of the world. I don't think this forced LTCM to liquidate positions early though, because LTCM used so little of its working capital due to financing trades through repo markets. Dealers were also hesitant to rollover short-term repo market agreements in bad states of the world, potentially restricting LTCM's financing ability. Equity redemption is the risk that investors might withdraw capital during periods of poor performance, although LTCM imposed a capital lock-up requirement of three years and allowed outflows on a staggered schedule. Lastly, as LTCM's positions deteriorated, financial institutions closed previously existing lines of credit and restricted loan access, further hampering LTCM's ability to meet collateral requirements.

4. LTCM is largely in the business of selling liquidity and volatility. Describe how LTCM accounts for liquidity risk in their quantitative measurements.

LTCM measures risk using the probability distribution of potential profits and losses like VaR, rare event stress testing and measuring the correlation of profits. LTCM accounts for liquidity risk by estimating how price movements can impact haircuts, or the cost of financing positions during periods of severe stress. This led to a schedule of potential worst-case liquidity demands over time, which helped LTCM to avoid forced liquidation of convergence positions.

5. Is leverage risk currently a concern for LTCM?

No, LTCM was 'only' leveraged 19:1 during the case study, although after returning $2.7 billion to investors (without reducing the scale of its investments) in 1997, the fund's leverage correspondingly returned to its historic high of 30:1.

6. What is the risk of convergence trades, where LTCM relies on converging spreads (if spreads converge, they make money or if it diverges, the trade becomes more attractive as convergence is still expected at a future data)?

The primary risk of convergence trades for LTCM was that the cost of financing could become too high, potentially forcing the fund to liquidate its positions before the expected price convergence could be achieved, resulting in significant losses. This risk was exacerbated by LTCM's high leverage and reliance on short-term financing, which made the fund vulnerable to margin calls and creditor demands during periods of market stress.

### 2. LTCM Risk Decomposition

In [1]:
# Set-up & Import data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import warnings

import helper_funcs

warnings.filterwarnings("ignore")
sns.set_theme()

# import helper functions
import importlib
import helper_funcs as hf

# Import data
ltcm = pd.read_excel('data/clean_data.xlsx').set_index('date')

#### 2.1 Summary Stats

1. For both the gross and net series of LTCM excess returns, report the mean, volatility, and Sharpe ratios. (Annualize them.)
2. Report the skewness, kurtosis, and (historic) VaR(.05)
3. Comment on how these stats compare to SPY and other assets we have seen. How much do they differ between gross and net?

LTCM's gross returns are significantly better (7% more average return per year, higher Sharpe ratio) than SPY during this period. The downside risk, as measured by excess kurtosis, VaR, and max drawdown, shows that LTCM and SPY have similar risk profiles, although note that SPY was well-behaved (nearly monotonically increasing) during this period. The net returns, gross returns less fees, are more aligned with the market.

In [2]:
importlib.reload(helper_funcs)

# Calc excess returns
ltcm_excess_ret = ltcm.sub(ltcm['rf'] / 12, axis=0)
ltcm_excess_ret.drop('rf', axis=1, inplace=True)

# Calc summary stats
summary_stats = hf.performance_summary(ltcm_excess_ret).round(4)
display(summary_stats[['Mean', 'Volatility', 'Sharpe Ratio', 'Skewness', 'Excess Kurtosis', 'VaR (0.05)', 'Max Drawdown']])

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),Max Drawdown
spy,0.1752,0.1127,1.5551,-0.4325,-0.3587,-0.0466,-0.0559
gross_ret,0.2433,0.1363,1.7858,-0.2897,1.5800,-0.0304,-0.1688
net_ret,0.1566,0.1118,1.4009,-0.8117,2.9172,-0.0264,-0.1754


#### 2.2 Estimate regression of net LTCM excess returns against market return

1. Report alpha, beta, and R-squared.
2. Does LTCM appear to be a "closet indexer"?
3. Does LTCM appear to deliver excess returns beyond the market risk premium?

In [3]:
# Alpha
model = sm.OLS(ltcm_excess_ret[['net_ret']], sm.add_constant(ltcm_excess_ret['spy'])).fit()
intercept = model.params[0].round(3)  # Intercept is the first parameter
print(f"Annualized intercept: {intercept * 12}")

# Market Beta
beta_first_coefficient = model.params[1].round(3)  # First coefficient (excluding intercept)
print(f"Market beta: {beta_first_coefficient}")

# R-squared value
r_squared = model.rsquared.round(3)
print(f"R-squared value: {r_squared}")


Annualized intercept: 0.132
Market beta: 0.137
R-squared value: 0.019


#### 2.3 Check for non-linear market exposure

1. Report betas and R-squared.
2. Does the quadratic market factor increase the overall LTCM variation explained by the market?
3. Does LTCM's market exposure behave as if it long or short market options?
4. Should we describe LTCM as being positively or negatively exposed to market volatility?

In [4]:
# Non-linear regression
spy = ltcm_excess_ret['spy']
spy_squared = spy ** 2
X = pd.DataFrame({
    'constant': 1,
    'spy': spy,
    'spy_squared': spy_squared
})
model = sm.OLS(ltcm_excess_ret[['net_ret']], X).fit()

# Analysis
# Market Beta
beta1 = model.params[1].round(3)
print(f"Market beta: {beta1}")
beta2 = model.params[2].round(3)
print(f"Quadratic market beta: {beta2}")

# R-squared value
r_squared = model.rsquared.round(3)
print(f"R-squared value: {r_squared}")


Market beta: 0.167
Quadratic market beta: -1.929
R-squared value: 0.025


#### 2.4. Does LTCM's nonlinear exposure come from up-markets or down-markets?

1. Report regression coefficients and R-squared value.
2. Is LTCM long or short the call-like factor? And the put-like factor?
3. Which factor moves LTCM more, the call-like factor or the put-like factor?
4. In section 3, we analyze if LTCM is positively or negatively exposed to market volatility. Does this volatility exposure come more from being long the market's update? Short the market's downside? Or something else?

In [5]:
# Call/ put regression
k1 = 0.03
k2 = -0.03
spy = ltcm_excess_ret['spy']
spy_call = spy - k1
spy_put = k2 - spy
spy_squared = spy ** 2
X = pd.DataFrame({
    'constant': 1,
    'spy': spy,
    'spy_call': spy_call.apply(lambda x: max(x, 0)),
    'spy_put': spy_put.apply(lambda x: max(x, 0))
})
model = sm.OLS(ltcm_excess_ret[['net_ret']], X).fit()

# Analysis
# Market Beta
beta1 = model.params[1].round(4)
beta2 = model.params[2].round(4)
beta3 = model.params[3].round(4)
print(f"Market beta: {beta1}\n"
      f"Call beta: {beta2}\n"
      f"Put beta: {beta3}\n")

# R-squared value
r_squared = model.rsquared.round(4)
print(f"R-squared value: {r_squared}")

Market beta: 0.4729
Call beta: -0.7897
Put beta: 1.3034

R-squared value: 0.0566


### 3. The FX Carry Trade

#### 3.1. Static Carry Trade

1. Report the mean, volatility, and Sharpe ratio of the excess log return relative to USD for each foreign currency. What differences do you see across currencies?



In [6]:
# Data import
rf = pd.read_excel('data/fx_rf_data.xlsx', sheet_name='risk-free rates').set_index('date')
fx = pd.read_excel('data/fx_rf_data.xlsx', sheet_name='exchange rates').set_index('date')

# Data cleaning
log_rf = rf.apply(lambda x: np.log(1 + x))
log_fx = fx.apply(lambda x: np.log(x))

# Static carry trade for each currency
currencies = ['JPY', 'EUR', 'GBP', 'MXN', 'CHF']
carry_excess_ret = pd.DataFrame(index=log_fx.index)
for currency in currencies:
    tmp_fx = log_fx[currency] - log_fx[currency].shift(1)
    tmp_rf = log_rf[currency].shift(1) - log_rf['USD'].shift(1) # rf rate from t -> t+1 known at time t
    carry_excess_ret[currency] = tmp_fx + tmp_rf

summary_stats = hf.performance_summary(carry_excess_ret).round(4)
display(summary_stats[['Mean', 'Volatility', 'Sharpe Ratio']])

,Mean,Volatility,Sharpe Ratio
JPY,-0.1405,0.0591,-2.3789
EUR,-0.0814,0.0433,-1.8807
GBP,-0.0021,0.0355,-0.0584
MXN,0.5517,0.0462,11.9477
CHF,-0.1541,0.0505,-3.0522


#### 3.2 Implications for the Uncovered Interest Parity (UIP)

1. Do any of these stats contradict the UIP?

L8.20, if UIP holds, carry trade should generate no excess return.

2. A long position in which foreign currency offered the best Sharpe ratio over the sample?
3. Are there any foreign currencies for which a long position earned a negative excess return in USD over the sample?

#### 3.3 Predicting FX

1. Report regression statistics (intercept, beta, R-squared) for each currency regression.
2. Suppose the foreign risk-free rate increases relative to the US rate.
    1. For which foreign currencies would we predict a relative strengthening of the USD in the following period?
    2. For which currencies would we predict relative weakening of the USD in the following period?

    See note at end of page 4 for interpretation.

    3. This FX predictability is strongest in the case of which foreign currency?

In [7]:
# Regression for each currency
currencies = ['JPY', 'EUR', 'GBP', 'MXN', 'CHF']
currency_reg_stats = pd.DataFrame(index=['Intercept', 'Beta', 'R-squared'])
for currency in currencies:
    y = log_fx[currency] - log_fx[currency].shift(1) # all time-t variables lagged
    x = log_rf['USD'].shift(1) - log_rf[currency].shift(1)
    tmp_reg_data = pd.DataFrame({'x' : x, 'y': y})
    tmp_reg_data['const'] = 1
    tmp_reg_data.dropna(inplace=True)

    model = sm.OLS(tmp_reg_data['y'], tmp_reg_data[['const','x']]).fit()

    # save results
    tmp_reg_stats = pd.Series([model.params['const'], model.params['x'], model.rsquared],
                              index=['Intercept', 'Beta', 'R-squared'])
    currency_reg_stats[currency] = tmp_reg_stats

display(currency_reg_stats)

,JPY,EUR,GBP,MXN,CHF
Intercept,-0.000087,-0.000124,-0.000100,-0.000313,0.000016
Beta,0.001452,0.008577,0.027484,-0.003782,0.003575
R-squared,0.000014,0.000288,0.001517,0.000023,0.000047


#### 3.4 Dynamic Carry Trade

1. Calculate how often the estimated FX risk premium is positive.
2. Which currencies most consistently have a positive FX risk premium? And for which currencies does the FX risk premium most often go negative?
3. Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.


In [8]:
# Estimated the FX risk premium
print('Percentage of periods (days) with positive FX risk premium')
for currency in currencies:
    int_rate_diff = log_rf['USD'].shift(1) - log_rf[currency].shift(1)
    est_fx_premium = currency_reg_stats[currency]['Intercept'] + (currency_reg_stats[currency]['Intercept'] - 1)*int_rate_diff

    # Calculate the percentage of positive numbers
    positive_count = (est_fx_premium > 0).sum()
    total_count = len(est_fx_premium)
    percent_positive = (positive_count / total_count) * 100

    # Display the result
    print(f"{currency}: {percent_positive:.2f}%")


Percentage of periods (days) with positive FX risk premium
JPY: 0.09%
EUR: 25.97%
GBP: 52.08%
MXN: 99.98%
CHF: 2.64%
